In [ ]:
# --- SEKCJA IMPORTÓW ---
# Importowanie niezbędnych bibliotek.

import os  # Biblioteka do interakcji z systemem operacyjnym, np. do zarządzania ścieżkami plików.
import re  # Biblioteka do obsługi wyrażeń regularnych, używana do czyszczenia tekstu.
import numpy as np  # Podstawowa biblioteka do obliczeń numerycznych, szczególnie na tablicach (macierzach).
import pandas as pd  # Biblioteka do manipulacji i analizy danych, głównie do wczytywania plików CSV.
import gensim  # Biblioteka do modelowania tematycznego i przetwarzania języka naturalnego, używana do Word2Vec.
import torch  # Główna biblioteka do deep learningu (PyTorch), na której budujemy nasz model.
import torch.nn as nn  # Moduł PyTorcha zawierający warstwy sieci neuronowych (np. LSTM, Linear).
import torch.optim as optim  # Moduł PyTorcha zawierający algorytmy optymalizacyjne (np. Adam).
from torch.utils.data import Dataset, DataLoader  # Narzędzia PyTorcha do tworzenia własnych zbiorów danych i ładowania ich w partiach.
from sklearn.model_selection import train_test_split  # Funkcja z scikit-learn do podziału danych na zbiór treningowy i testowy.
import time  # Biblioteka do mierzenia czasu wykonania operacji.

# Importy z TensorFlow/Keras – używane tylko do tokenizacji i paddingu, bo są bardzo wygodne.
from tf_keras.preprocessing.text import Tokenizer  # Narzędzie Keras do zamiany słów na unikalne identyfikatory (liczby).
from tf_keras.preprocessing.sequence import pad_sequences  # Narzędzie Keras do wyrównywania długości sekwencji (recenzji).
import kagglehub  # Biblioteka do łatwego pobierania zbiorów danych bezpośrednio z platformy Kaggle.

# Ukrycie mniej ważnych komunikatów od TensorFlow/oneDNN, aby nie zaśmiecały wyjścia konsoli.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# --- KONFIGURACJA STAŁYCH (HIPERPARAMETRY) ---
# Ustawienie kluczowych parametrów, które będą sterować procesem uczenia modelu.

MAX_SEQUENCE_LENGTH = 300   # Maksymalna długość recenzji w słowach. Dłuższe będą ucinane, krótsze uzupełniane zerami.
EMBEDDING_DIM = 100         # Wymiar wektora dla każdego słowa w modelu Word2Vec (każde słowo będzie reprezentowane przez 100 liczb).
W2V_MIN_COUNT = 5           # Minimalna częstotliwość występowania słowa w korpusie, aby zostało uwzględnione w modelu Word2Vec.
LSTM_HIDDEN_DIM = 128       # Liczba neuronów w warstwie ukrytej sieci LSTM.
NUM_CLASSES = 2             # Liczba klas wyjściowych (1 dla sentymentu pozytywnego, 0 dla negatywnego).
NUM_LAYERS = 3              # Liczba warstw LSTM ułożonych jedna na drugiej (stacked LSTM).
DROPOUT = 0.3               # Prawdopodobieństwo "wyłączenia" neuronu podczas treningu, technika regularyzacji zapobiegająca przeuczeniu.
LEARNING_RATE = 0.001       # Współczynnik uczenia, określa jak "mocno" model koryguje swoje wagi w każdej iteracji.
EPOCHS = 5                  # Liczba pełnych przejść przez cały zbiór danych treningowych.
BATCH_SIZE = 64             # Liczba recenzji przetwarzanych jednocześnie w jednej iteracji treningowej.
TEST_SIZE = 0.2             # Procent danych, który zostanie przeznaczony na zbiór testowy (tutaj 20%).

# --- DEFINICJE KLAS PYTORCH (MODEL I OBSŁUGA DANYCH) ---

class Attention(nn.Module):
    """
    Implementacja mechanizmu uwagi (Attention), który pozwala modelowi skupić się na najważniejszych słowach w recenzji.
    """
    def __init__(self, hidden_dim):
        # Inicjalizator klasy, dziedziczy po bazowej klasie `nn.Module` z PyTorcha.
        super(Attention, self).__init__()
        # Definiujemy jedną warstwę liniową (w pełni połączoną), która nauczy się przypisywać "wagę" każdemu słowu.
        # Wejście ma rozmiar `hidden_dim`, wyjście to pojedyncza liczba. `bias=False` to drobna optymalizacja.
        self.attention_layer = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, lstm_outputs):
        # Metoda `forward` definiuje, jak dane przepływają przez warstwę.
        # `lstm_outputs` ma kształt: (rozmiar_batcha, długość_sekwencji, wymiar_ukryty).

        # Przepuszczamy wyjścia z LSTM przez naszą warstwę liniową, aby uzyskać surowe wagi uwagi dla każdego słowa.
        # `squeeze(2)` usuwa ostatni wymiar, który jest zbędny (zmienia kształt z [batch, seq, 1] na [batch, seq]).
        attention_weights = self.attention_layer(lstm_outputs).squeeze(2)

        # Używamy funkcji softmax, aby znormalizować wagi. Dzięki temu sumują się do 1,
        # co można interpretować jako procentowe znaczenie każdego słowa w sekwencji.
        soft_attention_weights = torch.softmax(attention_weights, dim=1)

        # Mnożymy wagi przez wyjścia LSTM, aby stworzyć "wektor kontekstu".
        # `bmm` to mnożenie macierzy dla batchy. Wektor ten to ważona suma stanów ukrytych,
        # gdzie większy wkład mają słowa z wyższą wagą uwagi.
        context_vector = torch.bmm(soft_attention_weights.unsqueeze(1), lstm_outputs).squeeze(1)

        # Zwracamy wektor kontekstu, który reprezentuje całą recenzję ze skupieniem na kluczowych słowach.
        return context_vector

class EmbeddingClassifier(nn.Module):
    """
    Główna architektura modelu: wektory słów (embeddings) -> LSTM -> Attention -> Klasyfikator.
    """
    def __init__(self, embedding_dim, lstm_hidden_dim, num_layers, num_classes, dropout=0.3):
        # Inicjalizator głównej klasy modelu.
        super(EmbeddingClassifier, self).__init__()

        # Definiujemy warstwę LSTM.
        # `input_size` to wymiar wektora wejściowego (nasz EMBEDDING_DIM).
        # `hidden_size` to liczba neuronów w warstwie ukrytej.
        # `num_layers` to liczba warstw LSTM.
        # `batch_first=True` oznacza, że wymiar batcha jest pierwszy w kształcie tensora wejściowego.
        # `dropout` dodaje regularyzację między warstwami LSTM.
        # `bidirectional=False` oznacza, że używamy jednokierunkowego LSTM (przetwarza sekwencję od początku do końca).
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=lstm_hidden_dim, num_layers=num_layers,
                              batch_first=True, dropout=dropout, bidirectional=False)

        # Inicjalizujemy naszą własną warstwę Attention, podając jej wymiar warstwy ukrytej LSTM.
        self.attention = Attention(lstm_hidden_dim)

        # Definiujemy ostatnią warstwę, w pełni połączoną (fully connected), która dokona ostatecznej klasyfikacji.
        # Przyjmuje wektor kontekstu z warstwy Attention i zwraca logity dla każdej z `num_classes`.
        self.fc = nn.Linear(lstm_hidden_dim, num_classes)

    def forward(self, x):
        # Definicja przepływu danych przez cały model.
        # `x` to batch danych wejściowych (wektory słów).
        # Przepuszczamy dane przez LSTM. Otrzymujemy `lstm_outputs` (stany ukryte dla każdego kroku czasowego)
        # oraz `hidden` i `cell` (ostatnie stany ukryte i komórki).
        lstm_outputs, (hidden, cell) = self.lstm(x)

        # Wyjścia z LSTM przekazujemy do naszej warstwy Attention, aby uzyskać wektor kontekstu.
        context_vector = self.attention(lstm_outputs)

        # Wektor kontekstu jest przekazywany do warstwy klasyfikującej w celu uzyskania ostatecznych predykcji.
        out = self.fc(context_vector)
        # Zwracamy wynik klasyfikacji.
        return out

class TextEmbeddingDataset(Dataset):
    """
    Niestandardowy Dataset PyTorcha, który w locie zamienia sekwencje ID słów na gotowe wektory Word2Vec.
    """
    def __init__(self, sequences_padded, labels, embedding_matrix):
        # Inicjalizator Datasetu.
        self.sequences = sequences_padded  # Przechowuje dopełnione sekwencje ID słów.
        self.labels = torch.LongTensor(labels)  # Przechowuje etykiety jako tensory PyTorcha typu Long.
        # Przechowuje macierz embeddingów (wektorów słów) jako tensor PyTorcha.
        self.embedding_matrix = torch.from_numpy(embedding_matrix)

    def __len__(self):
        # Metoda wymagana przez PyTorch; zwraca całkowitą liczbę próbek w zbiorze danych.
        return len(self.sequences)

    def __getitem__(self, idx):
        # Metoda wymagana przez PyTorch; pobiera jedną próbkę (recenzję) na podstawie jej indeksu `idx`.
        indices = self.sequences[idx]  # Pobieramy sekwencję ID słów dla danej recenzji.

        # KLUCZOWY KROK: Używamy `torch.index_select`, aby wybrać wektory z `embedding_matrix`
        # odpowiadające indeksom (ID słów) w naszej sekwencji. To zamienia listę liczb na listę wektorów.
        embeddings = torch.index_select(self.embedding_matrix, 0, torch.LongTensor(indices))

        # Zwracamy parę: tensor z wektorami słów (dane wejściowe `X`) i etykietę (`Y`).
        return embeddings, self.labels[idx]

# --- KROK 1: POBIERANIE I WCZYTYWANIE DANYCH ---
print("pobieranie zbioru danych IMDb z Kaggle Hub...")
# Używamy biblioteki kagglehub do pobrania zbioru danych. Zwraca ona ścieżkę do pobranego folderu.
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
# Tworzymy pełną ścieżkę do pliku CSV wewnątrz pobranego folderu.
csv_file_path = os.path.join(path, "IMDB Dataset.csv")
# Wczytujemy plik CSV do obiektu DataFrame biblioteki pandas.
df = pd.read_csv(csv_file_path)
print("pobieranie i wczytywanie zakończone.")
print(f"wczytano {len(df)} recenzji.")

# Wyodrębniamy tekst recenzji do listy.
all_reviews_text = df['review'].tolist()
# Zamieniamy etykiety tekstowe ('positive', 'negative') na liczbowe (1, 0) za pomocą funkcji lambda.
all_labels = df['sentiment'].apply(lambda x: 1 if x == 'positive' else 0).values

# Dzielimy dane na zbiór treningowy i testowy w proporcji 80/20.
# `random_state=42` zapewnia powtarzalność podziału.
# `stratify=all_labels` zapewnia, że proporcje klas (pozytywnych/negatywnych) będą takie same w obu zbiorach.
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    all_reviews_text, all_labels, test_size=TEST_SIZE, random_state=42, stratify=all_labels)

# --- KROK 2: TRENOWANIE WORD2VEC (TWORZENIE EMBEDDINGÓW) ---
# Prosta tokenizacja tekstu na potrzeby gensim: usuwamy tagi HTML, znaki inne niż litery, zamieniamy na małe litery.
tokenized_corpus = [
    re.sub(r'[^a-z\s]', '', re.sub(r'<br />', ' ', review).lower()).split()
    for review in all_reviews_text]

print("rozpoczynanie trenowania modelu Word2Vec...")
start_w2v_time = time.time()  # Zapisujemy czas rozpoczęcia treningu Word2Vec.
# Trenujemy model Word2Vec na naszym tokenizowanym korpusie.
w2v_model = gensim.models.Word2Vec(
    sentences=tokenized_corpus,   # Dane wejściowe.
    vector_size=EMBEDDING_DIM,    # Wymiar wektora słowa.
    window=5,                     # Maksymalna odległość między bieżącym a przewidywanym słowem w zdaniu.
    min_count=W2V_MIN_COUNT,      # Ignoruje słowa o niższej częstotliwości.
    workers=4,                    # Liczba wątków procesora do użycia.
    sg=1                          # Używamy algorytmu Skip-gram (przewiduje kontekst na podstawie słowa).
)
end_w2v_time = time.time()  # Zapisujemy czas zakończenia.
print(f"trenowanie Word2Vec zakończone. wymiar embeddingów: {EMBEDDING_DIM}")
print(f"czas trenowania Word2Vec: {(end_w2v_time - start_w2v_time):.2f} sekund.")

# --- KROK 3: PRZYGOTOWANIE DANYCH DO PYTORCH ---
keras_tokenizer = Tokenizer()  # Inicjalizujemy tokenizer z Keras.
# Budujemy słownik (indeks słów) na podstawie wszystkich tekstów recenzji.
keras_tokenizer.fit_on_texts(all_reviews_text)
word_index = keras_tokenizer.word_index  # Pobieramy utworzony słownik mapujący słowa na liczby.
VOCAB_SIZE = len(word_index) + 1  # Całkowity rozmiar słownictwa (+1, bo indeks 0 jest zarezerwowany dla paddingu).

# Tworzymy pustą macierz numpy, która będzie przechowywać wektory słów.
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM), dtype=np.float32)

# Wypełniamy macierz wektorami z naszego wytrenowanego modelu Word2Vec.
for word, i in word_index.items():
    if word in w2v_model.wv:  # Sprawdzamy, czy słowo istnieje w słowniku Word2Vec.
        embedding_matrix[i] = w2v_model.wv[word]  # Jeśli tak, przypisujemy jego wektor do macierzy.

# Zamieniamy teksty na sekwencje liczb (ID słów) za pomocą tokenizera.
X_train_sequences = keras_tokenizer.texts_to_sequences(X_train_text)
X_test_sequences = keras_tokenizer.texts_to_sequences(X_test_text)

# Wyrównujemy długość wszystkich sekwencji do MAX_SEQUENCE_LENGTH.
# `padding='post'` dodaje zera na końcu. `truncating='post'` ucina sekwencje od końca.
X_train_padded = pad_sequences(X_train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post')

# Ustawienie optymalizacji dla DataLoader, jeśli dostępne jest GPU.
pin_memory = torch.cuda.is_available()

# Tworzymy obiekty naszych niestandardowych Datasetów.
train_dataset = TextEmbeddingDataset(X_train_padded, Y_train, embedding_matrix)
test_dataset = TextEmbeddingDataset(X_test_padded, Y_test, embedding_matrix)
# Tworzymy obiekty DataLoader, które będą dostarczać dane do modelu w partiach (batchach).
# `shuffle=True` losowo miesza dane treningowe w każdej epoce.
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=pin_memory)

# --- KROK 4: FUNKCJE TRENINGU I EWALUACJI ---
def train_model(model, loader, criterion, optimizer, device):
    """Funkcja do trenowania modelu przez jedną epokę."""
    model.train()  # Ustawia model w tryb treningowy (włącza m.in. dropout).
    total_loss = 0  # Suma błędów (straty) z całej epoki.
    correct = 0  # Liczba poprawnie sklasyfikowanych próbek.
    total = 0  # Całkowita liczba próbek.

    if device.type == 'cuda':  # Jeśli używamy GPU...
        torch.cuda.empty_cache()  # Czyścimy pamięć podręczną GPU.

    for inputs, labels in loader:  # Iterujemy po partiach danych z DataLoadera.
        # Przenosimy dane (wektory i etykiety) na wybrane urządzenie (CPU lub GPU).
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zerujemy gradienty z poprzedniej iteracji (bardzo ważne w PyTorch).
        outputs = model(inputs)  # Przepuszczamy dane przez model, aby uzyskać predykcje (forward pass).
        loss = criterion(outputs, labels)  # Obliczamy błąd (stratę) między predykcjami a prawdziwymi etykietami.

        loss.backward()  # Obliczamy gradienty straty względem wag modelu (backward pass).
        optimizer.step()  # Aktualizujemy wagi modelu na podstawie obliczonych gradientów.

        total_loss += loss.item()  # Dodajemy stratę z bieżącej partii do sumy.
        _, predicted = torch.max(outputs.data, 1)  # Wybieramy klasę z najwyższym prawdopodobieństwem jako predykcję.
        total += labels.size(0)  # Zwiększamy licznik przetworzonych próbek.
        correct += (predicted == labels).sum().item()  # Zliczamy poprawne predykcje.

    avg_loss = total_loss / len(loader)  # Obliczamy średnią stratę na partię.
    accuracy = 100 * correct / total  # Obliczamy celność (accuracy) w procentach.
    return avg_loss, accuracy  # Zwracamy średnią stratę i celność.

def evaluate_model(model, loader, criterion, device):
    """Funkcja do ewaluacji wydajności modelu na zbiorze danych."""
    model.eval()  # Ustawia model w tryb ewaluacji (wyłącza m.in. dropout).
    total_loss = 0  # Suma błędów (straty).
    correct = 0  # Liczba poprawnych predykcji.
    total = 0  # Całkowita liczba próbek.

    with torch.no_grad():  # Wyłączamy obliczanie gradientów, co przyspiesza ewaluację i oszczędza pamięć.
        for inputs, labels in loader:  # Iterujemy po partiach danych.
            inputs, labels = inputs.to(device), labels.to(device)  # Przenosimy dane na odpowiednie urządzenie.

            outputs = model(inputs)  # Uzyskujemy predykcje z modelu.
            loss = criterion(outputs, labels)  # Obliczamy stratę.
            total_loss += loss.item()  # Sumujemy stratę.

            _, predicted = torch.max(outputs.data, 1)  # Wybieramy przewidywaną klasę.
            total += labels.size(0)  # Zliczamy próbki.
            correct += (predicted == labels).sum().item()  # Zliczamy poprawne predykcje.

    avg_loss = total_loss / len(loader)  # Obliczamy średnią stratę.
    accuracy = 100 * correct / total  # Obliczamy celność.
    return avg_loss, accuracy  # Zwracamy wyniki.

# --- KROK 5: INICJALIZACJA I PĘTLA TRENINGOWA ---
# Sprawdzamy, czy dostępne jest GPU (CUDA) i ustawiamy odpowiednie urządzenie.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"używane urządzenie: {device}")

# Tworzymy instancję naszego modelu, przekazując zdefiniowane hiperparametry.
model = EmbeddingClassifier(
    embedding_dim=EMBEDDING_DIM,
    lstm_hidden_dim=LSTM_HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    num_classes=NUM_CLASSES,
    dropout=DROPOUT,
   ).to(device)  # Przenosimy cały model na wybrane urządzenie (GPU/CPU).

# Definiujemy funkcję straty. CrossEntropyLoss jest standardem dla problemów klasyfikacyjnych.
criterion = nn.CrossEntropyLoss()
# Definiujemy optymalizator. Adam jest popularnym i skutecznym wyborem.
# `model.parameters()` przekazuje wszystkie wagi modelu do optymalizatora.
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("\nrozpoczynanie treningu PyTorch LSTM...")
start_lstm_time = time.time()  # Zapisujemy czas rozpoczęcia całej pętli treningowej.

# Główna pętla treningowa, która iteruje przez zadaną liczbę epok.
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()  # Mierzymy czas startu epoki.

    # Uruchamiamy funkcję treningową na zbiorze treningowym.
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device)
    # Uruchamiamy funkcję ewaluacyjną na zbiorze testowym, aby sprawdzić, jak model generalizuje.
    test_loss, test_acc = evaluate_model(model, test_loader, criterion, device)

    epoch_end_time = time.time()  # Mierzymy czas końca epoki.
    epoch_duration = epoch_end_time - epoch_start_time  # Obliczamy czas trwania epoki.

    # Wyświetlamy wyniki dla bieżącej epoki.
    print(f"epoka {epoch}/{EPOCHS} (czas trwania: {epoch_duration:.2f} s)")
    print(f"  trening: loss={train_loss:.4f}, acc={train_acc:.2f}%")
    print(f"  test:    loss={test_loss:.4f}, acc={test_acc:.2f}%")

end_lstm_time = time.time()  # Zapisujemy czas zakończenia całego treningu.
total_lstm_time = end_lstm_time - start_lstm_time  # Obliczamy całkowity czas treningu.
print("trening PyTorch zakończony.")
print(f"całkowity czas treningu LSTM (dla {EPOCHS} epok): {total_lstm_time:.2f} sekund.")

# --- KROK 6: ZAPISANIE WYTRENOWANEGO MODELU ---
# Definiujemy ścieżkę do pliku, w którym zapiszemy wagi modelu.
SAVE_PATH = 'imdb_sentiment_model_weights.pth'
# Zapisujemy tylko "słownik stanu" (state_dict) modelu, czyli nauczone wagi i biasy.
# Jest to zalecana praktyka, ponieważ jest bardziej elastyczna niż zapisywanie całego obiektu modelu.
torch.save(model.state_dict(), SAVE_PATH)
print(f"\nwagi modelu zostały pomyślnie zapisane w pliku: {SAVE_PATH}")